Méthode qui vérifie la validité du masque

In [41]:
import re 

def verifierIntegriteMasque(masque):
    masqueValide = True

    maskInString = str(masque)
    if (re.search('[a-zA-Z_]',maskInString)):
        masqueValide = False
        return masqueValide

    if(len(masque)!=4 or maskInString.replace(" ","")==""):
        masqueValide = False
        return masqueValide

    for octet in masque:
        if (octet>255 or octet <0):
            masqueValide = False
            return masqueValide
        
    x = range(1,4)
    for n in x:
        if(masque[n]>masque[n-1]):
            masqueValide = False
    return masqueValide


In [42]:
print("Test avec un masque négatif ==> ",verifierIntegriteMasque([255,255,255,-1]))
print("Test avec un masque incomplet ==> ",verifierIntegriteMasque([255]))
print("Test avec un masque vide ==> ",verifierIntegriteMasque(""))
print("Test avec des espaces ==> ",verifierIntegriteMasque(" "))
print("Test avec des lettres ==> ",verifierIntegriteMasque(["Bonjour",255,0,0]))
print("Test avec un masque incohérant 1 ==>",verifierIntegriteMasque([255,-1,255,0]))
print("Test avec un masque incohérant 2 ==>",verifierIntegriteMasque([300,255,255,0]))
print("Test avec un masque incohérant 3 ==>",verifierIntegriteMasque([128,254,255,65]))
print("Test avec un masque normal ==>",verifierIntegriteMasque([255,255,255,0]))

Test avec un masque négatif ==>  False
Test avec un masque incomplet ==>  False
Test avec un masque vide ==>  False
Test avec des espaces ==>  False
Test avec des lettres ==>  False
Test avec un masque incohérant 1 ==> False
Test avec un masque incohérant 2 ==> False
Test avec un masque incohérant 3 ==> False
Test avec un masque normal ==> True


Méthode renvoyant la notation cidr d'un masque

In [43]:
def getNotationCIDR(masque):

    compteur = 0

    for octet in masque:
        octetBinare = bin(int(octet))[2:]
        
        for bit in octetBinare:
            if(bit == '1'):
                compteur = compteur +1

    
    return compteur

In [44]:
print("Test masque cidr 1 ==> ",getNotationCIDR([255,255,255,0]))

Test masque cidr 1 ==>  24


Méthode renvoyant le masque classique depuis un masque cidr


In [45]:
import ipaddress

def getNormalMask(masqueCIDR):
    if(masqueCIDR<0 or masqueCIDR>32):
        return False
    masque = "0.0.0.0/"+str(masqueCIDR)
    masqueNormal = ipaddress.ip_network(masque,strict = False)
    resultat = str(masqueNormal.with_netmask).replace("0.0.0.0/","")
    return(resultat)

In [46]:
print("Test masque normal 1 ==> ",getNormalMask(24))
print("Test masque normal 2 ==> ",getNormalMask(32))
print("Test masque normal 3 ==> ",getNormalMask(33))
print("Test masque normal 4 ==> ",getNormalMask(0))

Test masque normal 1 ==>  255.255.255.0
Test masque normal 2 ==>  255.255.255.255
Test masque normal 3 ==>  False
Test masque normal 4 ==>  0.0.0.0


In [47]:
import re 

def verifierIntegriteIP(adresseIP):
    IPValide = True

    IPString = str(adresseIP)

    if (re.search('[a-zA-Z_]',IPString)):
        IPValide = False
        return IPValide

    if(len(adresseIP)!=4 or IPString.replace(" ","")==""):
        IPValide = False
        return IPValide

    for octet in adresseIP:
        if (octet>255 or octet <0):
            IPValide = False
            return IPValide

    return IPValide

In [48]:
ip = "0 0 0 0"

ip = ip.split()

print(ip)

print("Test avec une IP négatif ==> ",verifierIntegriteIP([255,255,255,-1]))
print("Test avec un IP incomplet ==> ",verifierIntegriteIP([255]))
print("Test avec un IP vide ==> ",verifierIntegriteIP(""))
print("Test avec des espaces ==> ",verifierIntegriteIP(" "))
print("Test avec des lettres ==> ",verifierIntegriteIP(["Bonjour",255,0,0]))
print("Test avec un IP incohérant 1 ==>",verifierIntegriteIP([255,-1,255,0]))
print("Test avec un IP incohérant 2 ==>",verifierIntegriteIP([300,255,255,0]))
print("Test avec un IP normal ==>",verifierIntegriteIP([255,255,255,0]))


['0', '0', '0', '0']
Test avec une IP négatif ==>  False
Test avec un IP incomplet ==>  False
Test avec un IP vide ==>  False
Test avec des espaces ==>  False
Test avec des lettres ==>  False
Test avec un IP incohérant 1 ==> False
Test avec un IP incohérant 2 ==> False
Test avec un IP normal ==> True


In [50]:
def TrouverAdresseSousReseau(ip,masque):
    resultat = ['0', '0', '0', '0']
    x = range(0,4)
    for n in x:
        resultat[n] = int(ip[n]) & int(masque[n])
    return resultat

In [51]:
def TrouverAdresseBroadcast(ip,masque):
    masqueCIDR = getNotationCIDR(masque)
    ipString = str(ip[0]) + "." + str(ip[1]) +"." + str(ip[2]) +"." + str(ip[3])
    ipn= ipaddress.ip_network(str(ipString)+"/"+str(masqueCIDR))
    return ipn.broadcast_address

In [70]:
def trouverAdresseReseau(ip,masque):
    indiceClass = 0
    for o in masque:
        if (o=='255'):
            indiceClass = indiceClass+1

    indiceMasque = 0
    ip[0] = int(ip[0])
    if(ip[0]==127 or ip[0]==0 or ip[0] >223):
        indiceMasque=0
    elif(ip[0]>=0 and ip[0]<=127):
        indiceMasque=1
    elif(ip[0]>=128 and ip[0]<=191):
        indiceMasque=2
    elif(ip[0]>=192 and ip[0]<=223):
        indiceMasque=3

    if(indiceMasque!=indiceClass):
        indiceMasque = 0

    if indiceMasque==0:
        return False

    ipReseau = ['0', '0', '0', '0']
    x = range(0,indiceMasque)
    for n in x:
        ipReseau[n] = str(ip[n])
    return ipReseau

In [69]:
def TrouverAdresseBroadcastClassfull(ip,masque):
    indiceClass = 0
    for o in masque:
        if (o=='255'):
            indiceClass = indiceClass+1

    indiceMasque = 0
    ip[0] = int(ip[0])
    if(ip[0]==127 or ip[0]==0 or ip[0] >223):
        indiceMasque=0
    elif(ip[0]>=0 and ip[0]<=127):
        indiceMasque=1
    elif(ip[0]>=128 and ip[0]<=191):
        indiceMasque=2
    elif(ip[0]>=192 and ip[0]<=223):
        indiceMasque=3

    if(indiceMasque!=indiceClass):
        indiceMasque = 0

    if indiceMasque==0:
        return False

    ipReseau = ['255', '255', '255', '255']
    x = range(0,indiceMasque)
    for n in x:
        ipReseau[n] = str(ip[n])
    return ipReseau


In [75]:
ip = ['100', '27', '45', '33']
masque = ['255', '0', '0', '0']


In [76]:
print(TrouverAdresseBroadcastClassfull(ip,masque))

print(trouverAdresseReseau(ip,masque))

['100', '255', '255', '255']
['100', '0', '0', '0']
